In [1]:
from PIL import Image, ImageDraw, ImageFont
from PIL.ImageOps import invert
import matplotlib.pyplot as plt
import numpy as np
import math
from math import sqrt
import os
# from numpy import mean
# from math import pow, log, log2, floor


In [2]:
def create_folder(dir, folder_name):

    output_dir = os.path.join(dir, folder_name)

    if not os.path.isdir(output_dir):
        os.mkdir(output_dir)
    return output_dir

In [3]:
def haralic_kernel_f1_d45(input_array:np.ndarray, path:str, k):
    
    height, width = input_array.shape[:2]
    height_exp, width_exp = height+2, width+2

    harlik_matrix = np.zeros((257, 257), dtype=int)

    #создание вспомогательной матрицы для полного обхода краев
    expand_input_array = np.zeros((height_exp, width_exp), dtype=int)

    for i in range (height_exp):
        for j in range  (width_exp):
            if i == 0 or i == height_exp-1 or j == 0 or j == width_exp-1:
                expand_input_array[i][j] = -1
            else:
                expand_input_array[i][j] = input_array[i-1][j-1]

    # обход
    indent_h = 1
    indent_w = 1

    for i in range (indent_h, height_exp-indent_h):
        for j in range (indent_w, width_exp-indent_w):
            harlik_matrix[expand_input_array[i][j]+1][expand_input_array[i-1][j-1]+1] +=1
            harlik_matrix[expand_input_array[i][j]+1][expand_input_array[i-1][j+1]+1] +=1
            harlik_matrix[expand_input_array[i][j]+1][expand_input_array[i+1][j-1]+1] +=1
            harlik_matrix[expand_input_array[i][j]+1][expand_input_array[i+1][j+1]+1] +=1

    # нормировка для отображения картинкой м отсечение вспомогательных краев
    harlik_matrix1 = harlik_matrix[1:257,
                                   1:257]
    maxim = np.max(harlik_matrix1)
    corr_coeff = maxim /255
    harlik_matrix2 = harlik_matrix1 // corr_coeff
    
    
    new_image = Image.fromarray(harlik_matrix2.astype(np.uint8))
    # path = create_folder(os.getcwd(),"output")
    new_image.save(os.path.join(path, f'haralik{k}.png'))
    
    
    aaaaaa = np.array(new_image)
    return aaaaaa

In [4]:
def calculate_corr(haralik: np.array):
    P_j = haralik.sum(axis=0)
    P_i = haralik.sum(axis=1)

    nu_j = np.array([i*item for i, item in enumerate(P_j)]).sum()
    nu_i = np.array([i*item for i, item in enumerate(P_i)]).sum()

    sigma_i = np.array([(j - nu_i)**2 * P_i[j] for j in range(haralik.shape[1])]).sum()
    sigma_j = np.array([(i - nu_j)**2 * P_j[i] for i in range(haralik.shape[0])]).sum()

    corr = 0

    for i in range(haralik.shape[0]):
        for j in range(haralik.shape[1]):
            corr += i*j*haralik[i,j]

    return (corr - nu_i*nu_j)/(np.sqrt(sigma_i)*np.sqrt(sigma_j))
    

In [5]:
def monochrome(input_array: np.array):
    height, width = input_array.shape[:2]
    output_array = np.zeros((height, width), dtype=input_array.dtype)

    for y in range(height):
        for x in range(width):
            output_array[y][x] = np.mean(input_array[y][x])

    return output_array
  

In [6]:
def contrast(img:np.array,L, path:str, k):
    flat_img = img.flatten()
    mn = round(np.mean(flat_img))

    positiveRange = max(2, max(flat_img) - mn)
    negativeRange = max(2, mn - min(flat_img))
    
    positiveAlpha = (2 ** (L-1))/ np.log(positiveRange)
    negativeAlpha = (2 ** (L-1))/ np.log(negativeRange)

    res_img = np.zeros_like(img)

    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            f = img[i, j] - mn
            if f >= 1:
                res_img[i, j] = mn + positiveAlpha * np.log(f)
            elif f <= -1:
                res_img[i, j] = mn - negativeAlpha * np.log(abs(f))
            else:
                res_img[i, j] = mn

    new_image = Image.fromarray(res_img.astype(np.uint8))
    new_image.save(os.path.join(path, f'contrast{k}.png'))
    
    return res_img

In [7]:
def main():
 
    DIR_input = create_folder(os.getcwd(),"input")
    DIR_output = create_folder(os.getcwd(),"output")
    DIR_haralik = create_folder(DIR_output, "haralik")
    DIR_CONTRAST = create_folder(DIR_output, "contrast")
    DIR_CONTRAST_haralik  = create_folder(DIR_output, "contrast_haralik")
    for i in range(1, 10):
        l = 4
        input_image = Image.open(os.path.join(DIR_input, f'test{i}.bmp'))
        input_array = np.array(input_image)

        new_array = monochrome(input_array)
        # print(new_array.shape[:])
        haralic = haralic_kernel_f1_d45(new_array, DIR_haralik, i)
        print(calculate_corr(haralic))
        contrast(new_array,l,  DIR_CONTRAST, i)


        haralic_kernel_f1_d45(contrast(new_array,l, DIR_CONTRAST, i),  DIR_CONTRAST_haralik, i)
        contrast( haralic_kernel_f1_d45 (contrast(new_array, l,DIR_CONTRAST, i),  DIR_CONTRAST_haralik, i),l,  DIR_CONTRAST_haralik, i)



if __name__ == '__main__':
    main()

-0.00022530070241831324
-2.9541853032124908e-05
-2.2975689755081948e-05
-0.00014246976203912938


C:\Users\Milana\AppData\Local\Temp\ipykernel_12424\3717664204.py:17: RuntimeWarning: invalid value encountered in sqrt
  return (corr - nu_i*nu_j)/(np.sqrt(sigma_i)*np.sqrt(sigma_j))


nan


C:\Users\Milana\AppData\Local\Temp\ipykernel_12424\3717664204.py:8: RuntimeWarning: overflow encountered in scalar multiply
  sigma_i = np.array([(j - nu_i)**2 * P_i[j] for j in range(haralik.shape[1])]).sum()
C:\Users\Milana\AppData\Local\Temp\ipykernel_12424\3717664204.py:9: RuntimeWarning: overflow encountered in scalar multiply
  sigma_j = np.array([(i - nu_j)**2 * P_j[i] for i in range(haralik.shape[0])]).sum()
C:\Users\Milana\AppData\Local\Temp\ipykernel_12424\3717664204.py:15: RuntimeWarning: overflow encountered in scalar add
  corr += i*j*haralik[i,j]


nan
-0.0003336519027854479
-0.15247401605160624
-0.000512520877090107
